In [1]:
import pandas as pd
import os
import joblib
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
!pip install user-agents
import user_agents

# путь к файлам, я подгружала их сразу в Colab
data_path = Path('./')

train_labels = pd.read_csv(data_path / 'train_labels.csv', sep=';')
#train_df = pd.read_csv(data_path / 'train.csv', sep=';')
test_df = pd.read_csv(data_path / 'test.csv', sep=';')
referer_vectors = pd.read_csv(data_path / 'referer_vectors.csv', sep=';')
geo_info = pd.read_csv(data_path / 'geo_info.csv', sep=';')
test_users = pd.read_csv(data_path / 'test_users.csv', sep=';')

def merge_csv_files(part1_filename="train_part1.csv", part2_filename="train_part2.csv", sep=';', **kwargs):
    try:
        df1 = pd.read_csv(part1_filename, sep=sep, **kwargs)
        df2 = pd.read_csv(part2_filename, sep=sep, **kwargs)
        merged_df = pd.concat([df1, df2], ignore_index=True)
        return merged_df

    except FileNotFoundError:
        print(f"Ошибка: Один или оба CSV-файла не найдены ({part1_filename}, {part2_filename}).")
        return None
    except pd.errors.EmptyDataError:
        print(f"Ошибка: Один или оба файла CSV пусты или содержат некорректные данные.")
        return None
    except Exception as e:
        print(f"Произошла ошибка при объединении CSV-файлов: {e}")
        return None


train_df = merge_csv_files("train_part1.csv", "train_part2.csv", sep=';')

# объединение train и test для создания признаков + добавление флага
all_data = pd.concat([train_df.assign(is_test=0), test_df.assign(is_test=1)], ignore_index=True)

# Объединение с geo_info и referer_vectors
all_data = all_data.merge(geo_info, on='geo_id', how='left')
all_data = all_data.merge(referer_vectors, on='referer', how='left')

print("Первоначальное объединение данных завершено. Размер all_data:", all_data.shape)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 7.0 MB/s eta 0:00:00
Первоначальное объединение данных завершено. Размер all_data: (908218, 24)


In [2]:
#БЛОК - ВЫДЕЛЕНИЕ ВРЕМЕННЫХ ПРИЗНАКОВ И ОБРАБОТКА USER_AGENT; выполнение - 7 мин


# преобразования request_ts в datetime
all_data['request_ts'] = pd.to_datetime(all_data['request_ts'], unit='s')

#временные признаки
all_data['hour'] = all_data['request_ts'].dt.hour
all_data['dayofweek'] = all_data['request_ts'].dt.dayofweek
all_data['month'] = all_data['request_ts'].dt.month
all_data['is_weekend'] = all_data['dayofweek'].isin([5, 6]).astype(int)


#парсинг user_agent, возвращение NaN для отсутсвующих значений и при ошибке
def parse_ua(ua_string):
    if pd.isna(ua_string):
        return [np.nan] * 5
    try:
        ua = user_agents.parse(ua_string)
        return [ua.os.family, ua.browser.family, ua.device.family, ua.is_mobile, ua.is_tablet]
    except Exception as e:
        return [np.nan] * 5


# Применяем функцию И создаем новые колонки
all_data[['os_family', 'browser_family', 'device_family', 'is_mobile', 'is_tablet']] = \
    all_data['user_agent'].apply(lambda x: pd.Series(parse_ua(x)))

# преобразование булевы признаки is_mobile и is_tablet в числовые (0 или 1)
all_data['is_mobile'] = all_data['is_mobile'].astype(float).fillna(0)
all_data['is_tablet'] = all_data['is_tablet'].astype(float).fillna(0)

In [ ]:
# БЛОК - ПРОДОЛЖЕНИЕ ВЫДЕЛЕНИЯ ПРИЗНАКОВ; выполнение - 9 мин

#обработка referer
def extract_domain_path(referer_url):
    if pd.isna(referer_url) or not isinstance(referer_url, str):
        return np.nan, np.nan
    parts = referer_url.split('/')
    if len(parts) > 2: # https://domain/path -> parts[2] = domain, parts[3] = path
        domain = parts[2]
        path = parts[3] if len(parts) > 3 else np.nan
        return domain, path
    return np.nan, np.nan

all_data[['domain_hashed', 'path_hashed']] = all_data['referer'].apply(lambda x: pd.Series(extract_domain_path(x)))



#все категориальные столбцы
categorical_cols_to_encode = [
    'geo_id',
    'country_id',
    'region_id',
    'timezone',
    'os_family',
    'browser_family',
    'device_family',
    'domain_hashed',
    'path_hashed'
]

for col in categorical_cols_to_encode:
    if col in all_data.columns:
        all_data[col] = all_data[col].fillna(f'MISSING_{col}')


        # Создаем новый столбец с закодированными значениями
        le = LabelEncoder()
        if all_data[col].nunique() > 0: # Проверяем, есть ли уникальные значения для кодирования
            all_data[f'{col}_encoded'] = le.fit_transform(all_data[col])
        else:
            all_data[f'{col}_encoded'] = -1  #если уникальных нет, то кодируем -1
    else:
        print(f"Внимание: Колонка '{col}' не найдена в all_data. Пропускаем кодирование.")



#Агрегированные признаки на уровне пользователя
user_features = all_data.groupby('user_id').agg(
    count_requests=('request_ts', 'size'), #общее кол-во запросов
    min_ts=('request_ts', 'min'), #мин время запроса
    max_ts=('request_ts', 'max'), #макс время запроса
    nunique_geo_id=('geo_id', 'nunique'), #кол-во уникальных geo_id
    nunique_referer=('referer', 'nunique'),
    nunique_domain_hashed=('domain_hashed', 'nunique'),
    nunique_path_hashed=('path_hashed', 'nunique'),
    nunique_os_family=('os_family', 'nunique'),
    nunique_browser_family=('browser_family', 'nunique'),
    #среднее и стандартное отклонение referer_vectors
    **{f'mean_comp{i}': (f'component{i}', 'mean') for i in range(10)},
    **{f'std_comp{i}': (f'component{i}', 'std') for i in range(10)},
    #мода для категориальных, среднее для булевых из user_agent
    most_frequent_os=('os_family_encoded', lambda x: x.mode()[0] if not x.mode().empty else -1), # -1  для неизвестного
    most_frequent_browser=('browser_family_encoded', lambda x: x.mode()[0] if not x.mode().empty else -1),
    avg_is_mobile=('is_mobile', 'mean'),
    avg_is_tablet=('is_tablet', 'mean'),
).reset_index()

#продолжительность активности пользователя в секундах
user_features['user_activity_span_seconds'] = (user_features['max_ts'] - user_features['min_ts']).dt.total_seconds()
user_features.drop(columns=['min_ts', 'max_ts'], inplace=True)


# Присоединение пола (target) к обучающим пользователям
train_merged = train_labels.merge(user_features, on='user_id', how='left')

#nестовый датасет c присоединенными признаками
test_merged = test_users.merge(user_features, on='user_id', how='left')

print("Генерация агрегированных признаков завершена.")
print("Размер train_merged:", train_merged.shape)
print("Размер test_merged:", test_merged.shape)

Внимание: Колонка 'timezone' не найдена в all_data. Пропускаем кодирование.
Генерация агрегированных признаков завершена.
Размер train_merged: (500000, 34)
Размер test_merged: (85000, 33)


In [ ]:
#БЛОК - ПОДГОТОВКА И ОБУЧЕНИЕ; выполнение - 14 минут

#x - все кроме user_id и target, y - target
X = train_merged.drop(columns=['user_id', 'target'])
y = train_merged['target']

X_test_final = test_merged.drop(columns=['user_id'])

#для LightGBM выравниваем число признаков
common_cols = list(set(X.columns) & set(X_test_final.columns))
X = X[common_cols]
X_test_final = X_test_final[common_cols]

#список категориальных признаков
categorical_features_for_lgbm = []
for col in X.columns:
    #просмотр закодированных категориальных признаков
    if col.endswith('_encoded'):
        X[col] = X[col].astype('category')
        if col in X_test_final.columns: #проверка на существование в тестовом наборе
            X_test_final[col] = X_test_final[col].astype('category')
        categorical_features_for_lgbm.append(col)

    elif X[col].dtype == 'bool':
        X[col] = X[col].astype('category')
        if col in X_test_final.columns:
            X_test_final[col] = X_test_final[col].astype('category')
        categorical_features_for_lgbm.append(col)
    elif pd.api.types.is_numeric_dtype(X[col]):
        #числовые признаки пропускаем, не делаем категориальными
        pass
    else:
        #для прочих типов, если появятся
        try:
            X[col] = X[col].astype('category')
            if col in X_test_final.columns:
                X_test_final[col] = X_test_final[col].astype('category')
            categorical_features_for_lgbm.append(col)
            print(f"Преобразована колонка '{col}' к типу 'category' (была {X[col].dtype}).")
        except Exception as e:
            print(f"Ошибка при преобразовании '{col}' в 'category': {e}")
            pass

print(f"Количество признаков для модели: {len(X.columns)}")


NFOLDS = 5 #кол-во фолдов для кросс-валидации
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42) #баланс разделения классов
oof_preds = np.zeros(X.shape[0]) #предсказания для оценки на тренировочной выборке
sub_preds = np.zeros(X_test_final.shape[0]) #предсказания на тестовой выборке

print(f"Начинаем обучение LightGBM с {NFOLDS}-кратной StratifiedKFold валидацией...")

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]

    lgb_clf = lgb.LGBMClassifier(objective='binary',
                                 metric='auc',
                                 n_estimators=2000,
                                 learning_rate=0.01,
                                 num_leaves=64,
                                 max_depth=-1,
                                 min_child_samples=20,
                                 subsample=0.7,
                                 colsample_bytree=0.7,
                                 random_state=42,
                                 n_jobs=-1,
                                 is_unbalance=True
                                )

    lgb_clf.fit(X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='auc',
                callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
                categorical_feature=[col for col in categorical_features_for_lgbm if col in X_train.columns]
               )

    oof_preds[valid_idx] = lgb_clf.predict_proba(X_val)[:, 1]
    sub_preds += lgb_clf.predict_proba(X_test_final)[:, 1] / folds.n_splits

    print(f"Fold {n_fold+1} завершен. AUC на валидации: {roc_auc_score(y_val, oof_preds[valid_idx]):.4f}")


cv_auc_score = roc_auc_score(y, oof_preds)
print(f"\nСредний CV AUC на всех фолдах: {cv_auc_score:.4f}")


Количество признаков для модели: 32
Начинаем обучение LightGBM с 5-кратной StratifiedKFold валидацией...
[LightGBM] [Info] Number of positive: 190784, number of negative: 209216
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.322055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5423
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.476960 -> initscore=-0.092225
[LightGBM] [Info] Start training from score -0.092225
Fold 1 завершен. AUC на валидации: 0.8839
[LightGBM] [Info] Number of positive: 190784, number of negative: 209216
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.122279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5425
[LightGBM] [Info] Number of data points in the train set: 400000, number of used featur

In [ ]:
#сохранение последней модели
model_filename = 'gender_prediction_model.joblib'
joblib.dump(lgb_clf, model_filename)
print(f"\nМодель сохранена в '{model_filename}'")


#приведение к int для получения чистых 0 и 1, порог 0,5
final_predictions = (sub_preds >= 0.5).astype(int)

test_result = pd.DataFrame({'user_id': test_users['user_id'], 'target': final_predictions})
test_result.to_csv('test_result.csv', index=False, sep=';')

print("Файл test_result.csv успешно создан.")
print("Предсказания сохранены в 'test_result.csv'.")
print("Готово!")


Модель сохранена в 'gender_prediction_model.joblib'
Файл test_result.csv успешно создан.
Предсказания сохранены в 'test_result.csv'.
Готово!
